In [10]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

In [19]:
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Labour.csv')

In [30]:
model = smf.ols(data=df, formula='np.log(output)~np.log(capital)+np.log(labour)+I(np.log(capital)**2)+I(np.log(labour)**2)').fit(cov_type='HC3')
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(output)   R-squared:                       0.855
Model:                            OLS   Adj. R-squared:                  0.854
Method:                 Least Squares   F-statistic:                     553.3
Date:                Sun, 07 May 2023   Prob (F-statistic):          1.30e-193
Time:                        00:34:49   Log-Likelihood:                -353.28
No. Observations:                 569   AIC:                             716.6
Df Residuals:                     564   BIC:                             738.3
Df Model:                           4                                         
Covariance Type:                  HC3                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -1.3039      0.493     -2.643      0.008      -2.271      -0.337
np.log(capital)             0.1831      0.029      6.215      0.000       0.125       0.241
np.log(labour)              0.5153      0.206      2.497      0.013       0.111       0.920
I(np.log(capital) ** 2)     0.0227      0.008      2.737      0.006       0.006       0.039
I(np.log(labour) ** 2)      0.0203      0.021      0.965      0.334      -0.021       0.061
==============================================================================
Omnibus:                      202.378   Durbin-Watson:                   1.925
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2899.488
Skew:                           1.152   Prob(JB):                         0.00
Kurtosis:                      13.816   Cond. No.                         270.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [5]:
# Для того, чтобы проверить наличие гетероскедастичности с помощью теста Бройша-Пагана,
# необходимо построить вспомогательную регрессию,
#зависимой переменной в которой будут квадраты остатков исходной регресси.
I(age**2)

$$e_i = y_i - \hat{y}_i$$
$$
e_i^2 = \gamma_0 + \gamma_1*totwrk+age+\gamma_2*I(age**2)+\gamma_3*male+\gamma_3*smsa+\gamma_4*south
$$

In [6]:
# квадраты остатков добавим в датафрейм, воспользовавшись методом '.resid', возвращающий значения остатков
df['res2']=model.resid**2
df['res']=model.resid

In [7]:
 sub_model= smf.ols(data=df, formula='res2~totwrk+age+I(age**2)+male+smsa+south').fit()

## Сформулируем проверямемую гипотезу, о характере зависимости дисперсии ошибок от факторов регресси
$$
H_0: \sigma_i^2 \equiv \sigma^2=f(\gamma_0),
$$
$$
H_1:\sigma_i^2 = f(\gamma_0 + z_{i1}\gamma_1 + z_{i2}*\gamma_2 + \dots + \gamma_{ip}\gamma_p).
$$
### Другими словами, значимость вспомогательной регресси в целом, т.е.
$$
H_0: \gamma_1=\gamma_2=\gamma_3=\gamma_4=\gamma_5=\gamma_6=0,
$$
$$
H_1:  \gamma_1^2+\gamma_2^2+\gamma_3^2+\gamma_4^2+\gamma_5^2+\gamma_6^2>0.
$$

### При справедливости нулевой гипотезы статистика $n*R_0^2 $ имеет распределение хи-квадрат:
$$
nR_0^2 \approx_{H_0} \chi^2_p
$$

In [8]:
St = np.round(len(df)*sub_model.rsquared,2)
St

8.31

In [9]:
Hi2 = stats.chi2.ppf(1-0.05,6)
Hi2

12.591587243743977

$$
nR_0^2 < \chi_p^2.
$$
### Нет оснований отвергнуть нулевую гипотезу. Тест указывает на гомоскедастичность.